# "Hello World" with Pyomo

This is an example of basic usage of Pyomo for solving a linear programming problem with several open-source solvers.

In [3]:
import pyomo.environ as pyo
import pandas as pd

In [4]:
def lp_demo(solver_name='glpk'):
    """
    Function solves the following linear programming problem using the specified solver:

    Maximize z = 3*x1 + 5*x2
    Subject to:
        x1 + 2*x2 <= 8
        3*x1 + 2*x2 <= 12
        x1, x2 >= 0
    """

    # Create a model
    model = pyo.ConcreteModel()
    
    # Define variables
    model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
    model.x2 = pyo.Var(domain=pyo.NonNegativeReals)
    
    # Define objective function
    model.objective = pyo.Objective(expr=3 * model.x1 + 5 * model.x2, sense=pyo.maximize)
    
    # Define constraints
    model.constraint1 = pyo.Constraint(expr=model.x1 + 2 * model.x2 <= 8)
    model.constraint2 = pyo.Constraint(expr=3 * model.x1 + 2 * model.x2 <= 12)
    
    # Define solver
    solver = pyo.SolverFactory(solver_name)
    result = solver.solve(model)
    
    # Store results in a dictionary
    results = {
        "solver": solver_name,
        "status": result.solver.status,
        "termination_condition": result.solver.termination_condition,
        "optimal_value": model.objective(),
        "x1": model.x1(),
        "x2": model.x2()
    }
    
    return results

In [5]:
results = []
for solver in ['glpk', 'clp', 'cbc', 'appsi_highs', 'ipopt']:
    results.append(lp_demo(solver))
results_df = pd.DataFrame(results)

In [6]:
results_df

,solver,status,termination_condition,optimal_value,x1,x2
0,glpk,ok,optimal,21.0,2.0,3.0
1,clp,ok,optimal,21.0,2.0,3.0
2,cbc,ok,optimal,21.0,2.0,3.0
3,appsi_highs,ok,optimal,21.0,2.0,3.0
4,ipopt,ok,optimal,21.0,2.0,3.0


## Sensitivity analysis

Toy production planning problem adopted from [Jeff Kantor's notebook](https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/02.02-Production-Model-Sensitivity-Analysis.html). Factory is making widget X and Y, X sells for $40 and Y for $30. Making widget X requires 1 hour of LaborA and 2 hours of LaborB, while widget Y requires 1 hour of LaborA and 1 hour of LaborB. LaborA is constrained to <= 80 hours, LaborB is constrained to <=100 hours. Demand for widget A is <= 40 units.

The corresponding linear programming problem can be formulated as follows:

Maximize \( Z = 40x + 30y \)

Subject to:

\[
\begin{align*}
x &\leq 40 \\
x + y &\leq 80 \\
2x + y &\leq 100 \\
x, y &\geq 0
\end{align*}
\]

We can access dual values ("shadow prices"), which tell by how much the objective value would increase (for maximization problems) if corresponding constraint was relaxed by 1 unit. In this case, we can tell that increasing available pool of LaborA is the best way to increase profit. Negative signs are due to optimization direction (negative => maximization).

In [9]:
def sensitivity_analysis(solver='glpk'):
    """
    Function solves the following linear programming problem using the specified solver:

    Maximize z = 40*x + 30*y
    Subject to:
        x <= 40
        x + y <= 80
        2*x + y <= 100
        x, y >= 0
    """

    # Create a model
    model = pyo.ConcreteModel()

    # for access to dual solution for constraints
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    
    # Define variables
    model.x = pyo.Var(domain=pyo.NonNegativeReals)
    model.y = pyo.Var(domain=pyo.NonNegativeReals)
    
    # Define objective function
    model.profit = pyo.Objective(expr=40 * model.x + 30 * model.y, sense=pyo.maximize)
    
    # Define constraints
    model.demand = pyo.Constraint(expr=model.x <= 40)
    model.laborA = pyo.Constraint(expr=model.x + model.y <= 80)
    model.laborB = pyo.Constraint(expr=2 * model.x + model.y <= 100)
    
    # Define solver
    solver = pyo.SolverFactory(solver)
    result = solver.solve(model)
    
    # Store results in a dictionary
    results = {
        "solver": solver,
        "status": result.solver.status,
        "termination_condition": result.solver.termination_condition,
        "optimal_profit": model.profit(),
        "x": model.x(),
        "y": model.y(),
        "dual_demand": model.dual[model.demand],
        "dual_laborA": model.dual[model.laborA],
        "dual_laborB": model.dual[model.laborB]
    }
    
    # Capture printed values in a dataframe
    sensitivity_results = []

    for c in [model.demand, model.laborA, model.laborB]:
        sensitivity_results.append({
            "Constraint": str(c),
            "Value": c(),
            "Lslack": c.lslack(),
            "Uslack": c.uslack(),
            "Dual": model.dual[c]
        })

    sensitivity_df = pd.DataFrame(sensitivity_results)
    sensitivity_df

    return results, sensitivity_df

In [10]:
results, sensitivity_df = sensitivity_analysis('cbc')

In [13]:
results

{'solver': <pyomo.solvers.plugins.solvers.CBCplugin.CBCSHELL at 0x23f3f4a18d0>,
 'status': <SolverStatus.ok: 'ok'>,
 'termination_condition': <TerminationCondition.optimal: 'optimal'>,
 'optimal_profit': 2600.0,
 'x': 20.0,
 'y': 60.0,
 'dual_demand': 0.0,
 'dual_laborA': -20.0,
 'dual_laborB': -10.0}

In [12]:
sensitivity_df

,Constraint,Value,Lslack,Uslack,Dual
0,demand,20.0,inf,20.0,0.0
1,laborA,80.0,inf,0.0,-20.0
2,laborB,100.0,inf,0.0,-10.0
